# 20181001 Exploring Non-linear inputs into Linear classifier

## Goal
* explore how applying non-linear functions and adding them as new features into linear classifier can result in interesting new classification capabilities
* prioritize biologically-replicable non-linearizations

## Approach
* Use 2D system (before additional features) for easier visualization
* Create classification function
* Plot classification in feature space

## Linear Classifier Model

#### Converting from weight / bias form to plottable form

To plot: need to isolate a single variable (get it as a function of all other variables)

#### 2D
Line in vectorized form:

$$\bar{w} \cdot \bar{x} - k = 0$$

#### 2D + computed features
$\bar{x_c}$ are computed features, and their weights: $\bar{w_c}$:

$$\bar{w} \cdot \bar{x} + \bar{w_c} \cdot \bar{x_c} - k = 0$$

#### In plottable form

$$x_1 = -\frac{w_0}{w_1}x_0 - \frac{\bar{w_c} \cdot \bar{x_c}}{w_1} + \frac{k}{w_1} $$

#### If the computed feature includes $x_1$
* then $x_1$ needs to be solved for explicitly in terms of $x_{1,c}$
* see below for calculated inverse functions for NOT gate



## Calculated functions

### NOT function
* output is log10 because that's whats being compared by the summation (the log transform)

$$y = \log_{10}(\alpha \frac{K_d}{K_d + x^n} + y_{min})$$

#### Inverse function (for if the calculated feature is $x_1$)
Start with:

$$x_{1,c} = \log_{10}(\alpha \frac{K_d}{K_d + x_1^n} + y_{min})$$

Inverted:

$$ x_1 = \bigg( \frac{\alpha K_d}{10^{x_{1,c}} - y_{min}} - K_d \bigg)^{1/n}  $$

Nevermind... just use implicit function

## Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import importlib

import sys
sys.path.append('./modules')
import nonlin_inputs as nlin_h

#Symbolic math libraries
from sympy import plot_implicit, symbols, Eq
%matplotlib notebook

In [ ]:
%matplotlib inline

In [ ]:
importlib.reload(nlin_h);

### Nonlinear features with NOT functions

Vectorized NOT function

In [ ]:
# Create NOT function object
not_fnc = nlin_h.Not_fnc(n = 3, kd = .01)

x = np.logspace(-2,1,1000)
fig, ax = plt.subplots()

ax.loglog(x,not_fnc.get_output(x))
#ax.semilogx()
plt.show()

#### Linear classifier without computed features

In [ ]:
w_bar = np.array([0.9,2])
bias = 1

# Create lin classifier object
lin_class = nlin_h.Lin_2d_classifier(w_bar,bias)

x_range = np.linspace(0,10,100)

fig, ax = plt.subplots(1,1)

lin_class.plot(ax,x_range)

ax.set_xlim(left = 0, right = 10)
ax.set_ylim(bottom = 0, top = 10)
ax.set_xlabel('x');
ax.set_ylabel('y');

#### Linear classifier (NOT in log space)

In [ ]:
import sympy as sp

In [ ]:
x,y = symbols('x y')
p1 = plot_implicit(Eq(sp.log(y,10) - sp.log(x,10), sp.log(1,10)),(x,.1,10),(y,.1,10))
ax.set_xscale('log')
ax.set_yscale('log')

#### Create a computed NOT feature for x1

Computed features passed to Classifier object should be in the form of a function with one input (the x value). Pass it the not function object get_output function.

In [ ]:
importlib.reload(nlin_h);

In [ ]:
w_bar = np.array([-.5,0.5])
bias = 1
not_fnc = nlin_h.Not_fnc(n = 3, kd = .01)

computed_fncs = [not_fnc.get_output,not_fnc.get_output] # one for each input
w_c = [1,0] # also one for each input

lin_class = nlin_h.Lin_2d_classifier(
    w_bar, bias, x_c_fncs = computed_fncs, w_c = w_c)

In [ ]:
x_range = np.logspace(-2,2,100)
fig, ax = plt.subplots(1,1)
lin_class.plot(ax,x_range);
ax.set_xlim(left = 0.01, right = 10)
ax.set_ylim(bottom = 0.01, top = 10)
ax.set_xscale('log')
ax.set_yscale('log')

### Plotting with implicit functions

In [ ]:
importlib.reload(nlin_h);

In [ ]:
w_bar = np.array([-0.5,.6])
bias = 0.2
not_fnc = nlin_h.Not_fnc(n = 3, kd=0.1)

computed_fncs = [not_fnc.get_output,not_fnc.get_output] # one for each input
w_c = [-1.5,1.5] # also one for each input

lin_class = nlin_h.Lin_2d_classifier(
    w_bar, bias, x_c_fncs = computed_fncs, w_c = w_c)

In [ ]:
x_range = [0.01,10]
y_range = [0.01,10]

params = {'title': 'Non-log'}
lin_class.plot_imp(ax,x_range,y_range, plt_type = 'area', param_dict = params);

# plot log-log
params = {'xscale': 'log', 'yscale': 'log', 'title': 'Log-log'}
lin_class.plot_imp(ax,x_range,y_range, plt_type = 'area', param_dict = params);

## Computationally "plastic" circuits

Goal:
* look at how having components that are modifiable in vivo will change the computation

Parameters to look at:
* bias is probably the easiest to modify in vivo

### Modifying Bias
* fix weights / NOT gate selections and NOT gate weights
* change bias value

In [ ]:
importlib.reload(nlin_h);

#### Linear inputs

In [ ]:
# Make classifiers
biases = np.linspace(1,5,10)
w_bar = np.array([-.5,0.5])

classifiers = []

for bias in biases:
    lin_class = nlin_h.Lin_2d_classifier(w_bar,bias)
    classifiers.append(lin_class)

In [ ]:
# Plot them
fig, axes = plt.subplots(len(biases),1,figsize =(4,20))

x_range = np.linspace(0,10,100)

i = 0
for classifier in classifiers:
    ax = axes[i]
    classifier.plot(ax,x_range)
    ax.set_xlim(0,10)
    ax.set_ylim(0,10)
    i = i + 1

#### Non-linear inputs

In [ ]:
# Create logspaced bias range
biases = np.logspace(-2,1,10)

# Set parameters that will be shared for all classifier variants
w_bar = np.array([-.5,0.5])
not_fnc = nlin_h.Not_fnc(n = 3, kd=0.1)
computed_fncs = [not_fnc.get_output,not_fnc.get_output] # one for each input
w_c = [-1.5,1.5] # also one for each input

# Create empty classifer list to store classifier objects
classifiers = []

# Create a unique classifier object for each bias value
for bias in biases:
    classifier = nlin_h.Lin_2d_classifier(w_bar, bias, x_c_fncs = computed_fncs, w_c = w_c)
    classifiers.append(classifier)

In [ ]:
# Plot them all
x_range = [0.01,10]
y_range = [0.01,10]

# Add plots to a list
plot_list = []

for classifier in classifiers:
    params = {'title': 'Bias = {:.3f}'.format(classifier.bias)}
    a_plot = classifier.plot_imp(ax,x_range,y_range, plt_type = 'area', param_dict = params);
    plot_list.append(a_plot);


### Results
* modifying bias can change the classifier circuit in interesting ways